In [4]:
from pyspark.sql import SparkSession
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import VectorAssembler
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.sql.functions import col
import numpy as np
import os

In [3]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317145 sha256=cf024f0e70871f854241aa792a9d3418123fa934e29177e1caaafb2dc7557a15
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [6]:
os.environ["SPARK_HOME"] = "/usr/local/lib/python3.10/dist-packages/pyspark"
os.environ["HADOOP_HOME"] = "C:/winutils"

# Creating spark session
spark = SparkSession.builder.appName("Crime Analytics").getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

# Loading the dataset
KCDPFinal = spark.read.format("csv").option("header", True).option("inferSchema", True).option("delimiter", ",").load("/content/drive/MyDrive/DBMS_project_final/Source Code/1-Datasets/KCPD_Crime_Data/KCcrimeForAnalytics.csv").withColumnRenamed("Firearm_Used_Flag", "label")
KCDPFinal

# Create vector assembler for feature columns
VAssembler = VectorAssembler(inputCols=KCDPFinal.columns[1:19], outputCol="features")
KCDPFinal = VAssembler.transform(KCDPFinal)

# Split the crime dataset into training and testing data sets
trainingData, testingData = KCDPFinal.select("label", "features").randomSplit([0.7, 0.3])

# Using the training set for the model traning
from pyspark.ml.classification import NaiveBayes
NaiveBayesModel = NaiveBayes()
model = NaiveBayesModel.fit(trainingData)

# Generate prediction from test dataset
CrimepredKC = model.transform(testingData)

# Evuluate the accuracy of the model
evaluator = MulticlassClassificationEvaluator()
accuracy = evaluator.evaluate(CrimepredKC)

# Show model accuracy
print("Accuracy:", accuracy)

Accuracy: 0.6241575016174253
